In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By

import pandas as pd
import time
import wget
import os
import glob

In [ ]:
df = pd.read_csv('out_submit.csv')

In [ ]:
driver = webdriver.Chrome()

In [ ]:
energies = []

In [ ]:
for link in df['Link']:
    print('Collecting data...', link)
    driver.get(link)
    time.sleep(3)
    element = driver.find_element(By.XPATH, '/html/body/center/table[3]/tbody/tr[2]/td[1]')
    energies.append(element.text)
    # Docking Score (1st model)

    top10_models = link + 'top10_models.tar.gz'
    wget.download(top10_models)
    os.system('tar -xvzf top10_models.tar.gz')
    os.system('rm top10_models.tar.gz')
    # Download and extract files containing the 10 best positions of each peptide

In [ ]:
driver.close()

In [ ]:
df['Energies'] = energies

In [ ]:
df.to_csv('results_dock.csv')